In [7]:
!pip install gradio scikit-learn pandas


import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00


In [9]:
df = pd.read_csv('/content/activity_diet_data_5000.csv')

In [ ]:
df.head()

,Activity,DietPreference,SleepHours,Age,RecommendedDiet
0,High,Vegan,6.6,24,High Protein
1,Low,Veg,5.0,43,Balanced
2,High,Vegan,4.8,32,High Protein
3,High,Veg,4.1,66,Balanced
4,Low,Vegan,7.9,59,Balanced


In [10]:
df = pd.get_dummies(df, columns=['Activity', 'DietPreference'])


In [11]:
X = df.drop('RecommendedDiet', axis=1)
y = df['RecommendedDiet']


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
model = RandomForestClassifier()
model.fit(X_train, y_train)


RandomForestClassifier()

In [1]:
base_meal_plans = {
    "Balanced": {
        "Veg": {
            "Breakfast": "Idli with sambar",
            "Lunch": "Roti, dal, sabzi",
            "Dinner": "Khichdi with curd"
        },
        "Non-Veg": {
            "Breakfast": "Egg bhurji with toast",
            "Lunch": "Chicken curry with rice",
            "Dinner": "Fish curry with roti"
        },
        "Vegan": {
            "Breakfast": "Vegan poha with peanuts",
            "Lunch": "Vegan rajma with rice",
            "Dinner": "Vegan pulao with tofu"
        }
    },
    "Low Carb": {
        "Veg": {
            "Breakfast": "Paneer bhurji",
            "Lunch": "Palak paneer",
            "Dinner": "Cauliflower rice with sabzi"
        },
        "Non-Veg": {
            "Breakfast": "Boiled eggs with cucumber",
            "Lunch": "Grilled chicken salad",
            "Dinner": "Chicken curry with sautéed veggies"
        },
        "Vegan": {
            "Breakfast": "Tofu scramble",
            "Lunch": "Vegan salad with beans",
            "Dinner": "Zucchini noodles with sauce"
        }
    },
    "High Protein": {
        "Veg": {
            "Breakfast": "Moong dal chilla",
            "Lunch": "Paneer tikka",
            "Dinner": "Soya curry with roti"
        },
        "Non-Veg": {
            "Breakfast": "Omelette with veggies",
            "Lunch": "Grilled chicken with eggs",
            "Dinner": "Egg curry with rice"
        },
        "Vegan": {
            "Breakfast": "Lentil pancakes",
            "Lunch": "Tofu stir fry",
            "Dinner": "Chickpea stew"
        }
    },
    "Mediterranean": {
        "Veg": {
            "Breakfast": "Poha with peanuts",
            "Lunch": "Rajma salad",
            "Dinner": "Veg couscous"
        },
        "Non-Veg": {
            "Breakfast": "Egg wrap",
            "Lunch": "Grilled fish salad",
            "Dinner": "Chicken couscous"
        },
        "Vegan": {
            "Breakfast": "Avocado toast",
            "Lunch": "Chickpea quinoa salad",
            "Dinner": "Vegan stew with millet"
        }
    }
}


In [5]:
def recommend_diet(activity, diet_pref, sleep_hours, age, health_conditions):
    input_data = {col: 0 for col in X.columns}
    input_data[f'Activity_{activity}'] = 1
    input_data[f'DietPreference_{diet_pref}'] = 1
    input_data['SleepHours'] = sleep_hours
    input_data['Age'] = age
    user_df = pd.DataFrame([input_data])

    prediction = model.predict(user_df)[0]
    meals = base_meal_plans.get(prediction, {}).get(diet_pref, {}).copy()

    # Modify meals based on health conditions
    if "Diabetes" in health_conditions:
        meals["Breakfast"] = "Oats with nuts (no sugar)"
        meals["Lunch"] = "Brown rice with dal and lauki"
        meals["Dinner"] = "Tofu curry with spinach"

    if "Hypertension" in health_conditions:
        meals["Lunch"] = "Low-salt dal with steamed veggies"
        meals["Dinner"] = "Ragi roti with bottle gourd curry"

    if "Cholesterol" in health_conditions:
        meals["Breakfast"] = "Fruit bowl with flax seeds"
        meals["Lunch"] = "Oats khichdi with curd"
        meals["Dinner"] = "Grilled tofu with mixed salad"

    if "PCOS" in health_conditions:
        meals["Breakfast"] = "Methi paratha with curd"
        meals["Lunch"] = "Bajra roti with palak paneer"
        meals["Dinner"] = "Quinoa khichdi with cucumber raita"

    if "Thyroid" in health_conditions:
        meals["Breakfast"] = "Sprouted moong salad"
        meals["Lunch"] = "Brown rice with lauki curry"
        meals["Dinner"] = "Ragi dosa with coconut chutney"

    if "Acidity / GERD" in health_conditions:
        meals["Breakfast"] = "Oats porridge with banana"
        meals["Lunch"] = "Steamed rice with bottle gourd and moong dal"
        meals["Dinner"] = "Vegetable upma + buttermilk"

    if "Anemia" in health_conditions:
        meals["Breakfast"] = "Dates + jaggery + boiled eggs"
        meals["Lunch"] = "Rajma with spinach rice"
        meals["Dinner"] = "Beetroot tikki with curd"

    if "Lactose Intolerance" in health_conditions:
        for key in meals:
            meals[key] = meals[key].replace("curd", "coconut curd").replace("milk", "soy milk")

    if age > 60:
        meals["Dinner"] = "Moong dal khichdi with plant curd" if "Lactose Intolerance" in health_conditions else "Moong dal khichdi with curd"

    if sleep_hours < 6:
        meals["Dinner"] += " + Warm turmeric almond milk"

    # Return an attractive formatted output
    meal_plan_text = f"""
        <div style="text-align:center;">
            <b style='color: #6a5acd;'>🧠 Recommended Diet Plan: {prediction}</b>
            <br><br>
            <b>🍳 Breakfast:</b> <i>{meals['Breakfast']}</i> 🍴
            <br>
            <b>🍛 Lunch:</b> <i>{meals['Lunch']}</i> 🍽️
            <br>
            <b>🌙 Dinner:</b> <i>{meals['Dinner']}</i> 🍷
            <br><br>
            <i><b>💡STAY HEALTHY💡</b><br></i>
        </div>
    """
    return meal_plan_text


In [14]:
def recommend_diet_gui():
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("<div style='text-align: center;'><h1 style='color: #ff6347;'>🥗Personalized Diet Planner🥗</h1></div>")
        gr.Markdown("<div style='text-align: center;'><p style='font-size: 18px;'>Customize your lifestyle and health to get daily Indian meal recommendations.</p>")

        with gr.Row():
            activity = gr.Dropdown(['Low', 'Medium', 'High'], label="🏃 Activity Level", interactive=True)
            diet_pref = gr.Dropdown(['Veg', 'Non-Veg', 'Vegan'], label="🍽️ Diet Preference", interactive=True)

        with gr.Row():
            sleep_hours = gr.Slider(0, 12, step=0.1, label="🛌 Sleep Hours")
            age = gr.Slider(12, 90, step=1, label="🎂 Age")

        health_conditions = gr.CheckboxGroup(
            ['Diabetes', 'Hypertension', 'Cholesterol', 'PCOD', 'Thyroid', 'Acidity / GERD', 'Anemia', 'Lactose Intolerance'],
            label="🩺 Health Conditions"
        )

        output = gr.HTML(label="Your Personalized Meal Plan")

        submit_btn = gr.Button("✅ Get Meal Plan")
        submit_btn.click(fn=recommend_diet, inputs=[activity, diet_pref, sleep_hours, age, health_conditions], outputs=output)

    return demo

# Launch enhanced GUI
recommend_diet_gui().launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fe929b01b4532164aa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
